In [3]:
##############################################################################
#                                                                            #
#  Code for the USENIX Security '22 paper:                                   #
#  How Machine Learning Is Solving the Binary Function Similarity Problem.   #
#                                                                            #
#  MIT License                                                               #
#                                                                            #
#  Copyright (c) 2019-2022 Cisco Talos                                       #
#                                                                            #
#  Permission is hereby granted, free of charge, to any person obtaining     #
#  a copy of this software and associated documentation files (the           #
#  "Software"), to deal in the Software without restriction, including       #
#  without limitation the rights to use, copy, modify, merge, publish,       #
#  distribute, sublicense, and/or sell copies of the Software, and to        #
#  permit persons to whom the Software is furnished to do so, subject to     #
#  the following conditions:                                                 #
#                                                                            #
#  The above copyright notice and this permission notice shall be            #
#  included in all copies or substantial portions of the Software.           #
#                                                                            #
#  THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND,           #
#  EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF        #
#  MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND                     #
#  NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE    #
#  LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION    #
#  OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION     #
#  WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.           #
#                                                                            #
#  Evaluation notebook                                                       #
#                                                                            #
##############################################################################

In [4]:
import json
import numpy as np
import os
import pandas as pd

from IPython.display import display
from collections import defaultdict
from sklearn import metrics

import matplotlib as mpl
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

### Utils functions

In [5]:
def process_df(df_pairs, df_similarity, is_pos):
    df_pairs = df_pairs.merge(
        df_similarity,
        how='left',
        left_on=['idb_path_1', 'fva_1',
                 'idb_path_2', 'fva_2'],
        right_on=['idb_path_1', 'fva_1', 'idb_path_2', 'fva_2'])

    if is_pos:
        # If positive pairs, the perfect similarity is 1
        df_pairs['gt'] = [1] * df_pairs.shape[0]
    else:
        # if negative pairs, the perfect similarity is 0
        df_pairs['gt'] = [-1] * df_pairs.shape[0]

    return df_pairs

### Utility functions for plotting

In [6]:
def plot_roc_values(df_pos_input, df_neg_input, plot=None, plt_cosine_sim=False):
    task_list = sorted(list(set(df_pos_input['db_type'])))
    result_list = list()

    for task in task_list:
        # Filter test data by test case
        df_pos = df_pos_input[df_pos_input['db_type'] == task]
        df_neg = df_neg_input[df_neg_input['db_type'] == task]

        if plt_cosine_sim:
            print("Processing task: {}".format(task))
            display(df_pos['cs'].hist(bins=200))
            display(df_neg['cs'].hist(bins=200, alpha=0.8))
            plt.show()
            continue

        # Get predicted and ground truth value
        pred_list = list(df_pos['cs'].values) + \
            list(df_neg['cs'].values)
        gt_list = list(df_pos['gt'].values) + list(df_neg['gt'].values)

        # Compute the ROC AUC
        fpr, tpr, thresholds = metrics.roc_curve(gt_list, pred_list)
        roc_auc = metrics.roc_auc_score(gt_list, pred_list)
        print('%20s - AUC = %0.2f' % (task, roc_auc))
        result_list.append([
            "%20s" % (task),
            "%0.2f" % (roc_auc)])

        if plot:
            plt.plot(fpr,
                     tpr,
                     linewidth=1.0,
                     label='AUC = %0.2f' % (roc_auc))

        if plot:
            plt.title("Task: {}".format(task))
            plt.legend(loc='lower right')
            plt.plot([0, 1], [0, 1], 'k--')
            plt.xlim([0, 1])
            plt.ylim([0, 1])
            plt.ylabel('True Positive Rate')
            plt.xlabel('False Positive Rate')
            # plt.savefig("output/{}split_{}.png".format(split, task), dpi=300)
            plt.show()

    return result_list

## Compute ranking metrics

In [7]:
def get_ranking(pos, neg_results):

    # Sort the negative results in descending mode
    neg_results = sorted(neg_results, reverse=True)

    # BUG FIX:
    if pos == 0:
        return len(neg_results) + 1

    # Find the first position where the pos element is >=
    for c, i in enumerate(neg_results):
        if pos >= i:
            return c +1

    # Otherwise return the last position +1 
    return len(neg_results) + 1

In [8]:
def compute_ranking_metrics(df_pos_ranking_input, df_neg_ranking_input):
    ranking_result_dict = defaultdict(list)
    TOP_LIST = [1] + list(range(5, 55, 5))
    task_list = list(set(df_pos_ranking_input['db_type']))

    for task in sorted(task_list):
        # Filter test data by task
        df_pos = df_pos_ranking_input[
            df_pos_ranking_input['db_type'] == task]
        df_neg = df_neg_ranking_input[
            df_neg_ranking_input['db_type'] == task]

        # display(df_pos['cs'].hist(bins=200))
        # display(df_neg['cs'].hist(bins=200, alpha=0.8))
        # plt.show()

        num_test_cases = df_pos.shape[0]

        # Compute the ranking for all the positive test cases
        ranking_list = list()
        for idx, group in df_neg.groupby(['idb_path_1', 'fva_1']):
            c1 = (df_pos['idb_path_1'] == idx[0])
            c2 = (df_pos['fva_1'] == idx[1])
            pos_pred = df_pos[c1 & c2]['cs'].values[0]
            neg_pred = sorted(group['cs'].values, reverse=True)
            rank = get_ranking(pos_pred, neg_pred)
            ranking_list.append(rank)

        # Compute the ranking list
        top_results = list()
        for tt in TOP_LIST:
            r = len([x for x in ranking_list if x <= tt])
            top_results.append(r)

        # MRR metric
        MRR_list = [1 / x if x <= 10 else 0 for x in ranking_list]
        MRR = sum(MRR_list) / len(MRR_list)

        # Save data in a temporary dictionary
        ranking_result_dict["task"].append(task)
        ranking_result_dict["num_test_cases"].append(num_test_cases)
        for tt, tr in zip(TOP_LIST, top_results):
            ranking_result_dict["cc_top_{}".format(tt)].append(tr)
        ranking_result_dict["MRR"].append(MRR)

    df_ranking = pd.DataFrame.from_dict(ranking_result_dict)
    return df_ranking

## NN results (similarity is already available)

In [14]:
!mkdir -p _output/Dataset-2

In [15]:
RESULTS_DIR = "../data/Dataset-2/"
OUTPUT_DIR = "_output/Dataset-2/"

global_results_tc0 = dict()
global_results_tc1 = dict()
global_results_tc2 = dict()

mrr_recall_tc0 = dict()
mrr_recall_tc1 = dict()
mrr_recall_tc2 = dict()

for csv_file in os.listdir(RESULTS_DIR):
    if (not csv_file.endswith(".csv")) or \
        (not csv_file.startswith("pos_")) or \
            (not "_rank_" in csv_file):
        continue

    print("Processing {}".format(csv_file))

    # Load data
    df_pos_ranking_input = pd.read_csv(
        os.path.join(RESULTS_DIR, csv_file))

    df_neg_ranking_input = pd.read_csv(
        os.path.join(RESULTS_DIR, csv_file.replace("pos_", "neg_")))

    if 'cs' not in set(df_pos_ranking_input.columns):
        df_pos_ranking_input.rename(columns={'sim': 'cs'}, inplace=True)
        df_neg_ranking_input.rename(columns={'sim': 'cs'}, inplace=True)

    # Plot the similarity values
    print(
        df_pos_ranking_input['cs'].mean(),
        df_neg_ranking_input['cs'].mean())

    # display(df_pos_ranking_input['cs'].hist(bins=200, range=[-10, 0]))
    # display(df_neg_ranking_input['cs'].hist(bins=200, alpha=0.8, range=[-10, 0]))
    # plt.show()

    # Check that similarity values are not 0
    assert(df_pos_ranking_input.isna().sum()['cs'] == 0)
    assert(df_neg_ranking_input.isna().sum()['cs'] == 0)

    rank_df = compute_ranking_metrics(df_pos_ranking_input, df_neg_ranking_input)
    key = csv_file.replace("pos_", "").replace("neg_", "").replace(".csv", "")

    global_results_tc0[key] = list(rank_df.loc[0][3:-1].values)
    global_results_tc1[key] = list(rank_df.loc[1][3:-1].values)
    global_results_tc2[key] = list(rank_df.loc[2][3:-1].values)

    rank_df["recall@1"] = rank_df['cc_top_1']/200
    display(rank_df)

    mrr_recall_tc0[key] = list(rank_df.loc[0][['MRR', "recall@1"]])
    mrr_recall_tc1[key] = list(rank_df.loc[1][['MRR', "recall@1"]])
    mrr_recall_tc2[key] = list(rank_df.loc[2][['MRR', "recall@1"]])

    print(key)
    display(rank_df.loc[0][['MRR', "recall@1"]])
    display(rank_df.loc[1][['MRR', "recall@1"]])
    display(rank_df.loc[2][['MRR', "recall@1"]])

    print("\n")

Processing pos_rank_testing_Dataset-2_SAFE_ASM-list_Trainable_e10.csv
0.6142685669880001 -0.5273025614075466


,task,num_test_cases,cc_top_1,cc_top_5,cc_top_10,cc_top_15,cc_top_20,cc_top_25,cc_top_30,cc_top_35,cc_top_40,cc_top_45,cc_top_50,MRR,recall@1
0,1,200,51,119,150,162,173,179,183,183,187,191,194,0.397887,0.255
1,2,200,49,134,169,184,190,192,192,195,197,197,198,0.426571,0.245
2,3,200,46,109,143,157,168,181,183,185,189,191,191,0.372282,0.230


rank_testing_Dataset-2_SAFE_ASM-list_Trainable_e10


MRR         0.397887
recall@1    0.255000
Name: 0, dtype: float64

MRR         0.426571
recall@1    0.245000
Name: 1, dtype: float64

...

,task,num_test_cases,cc_top_1,cc_top_5,cc_top_10,cc_top_15,cc_top_20,cc_top_25,cc_top_30,cc_top_35,cc_top_40,cc_top_45,cc_top_50,MRR,recall@1
0,XA,200,6,27,38,56,66,86,103,118,126,145,153,0.072512,0.030
1,XA+XO,200,39,53,73,85,98,111,123,132,138,146,154,0.229296,0.195
2,XO,200,100,159,176,182,188,189,190,193,195,196,197,0.625468,0.500


rank_testing_Dataset-2_pvdbow_e10


MRR         0.072512
recall@1        0.03
Name: 0, dtype: object

MRR         0.229296
recall@1       0.195
Name: 1, dtype: object

MRR         0.625468
recall@1         0.5
Name: 2, dtype: object

In [16]:
df_mrr_recall = pd.DataFrame(mrr_recall_tc0)
df_mrr_recall = df_mrr_recall.transpose()
df_mrr_recall = df_mrr_recall.rename(columns={0:"MRR", 1:"recall@1"})
df_mrr_recall.to_csv(os.path.join(OUTPUT_DIR, "df_mrr_recall_tc1.csv"))

df_mrr_recall = pd.DataFrame(mrr_recall_tc1)
df_mrr_recall = df_mrr_recall.transpose()
df_mrr_recall = df_mrr_recall.rename(columns={0:"MRR", 1:"recall@1"})
df_mrr_recall.to_csv(os.path.join(OUTPUT_DIR, "df_mrr_recall_tc2.csv"))

df_mrr_recall = pd.DataFrame(mrr_recall_tc2)
df_mrr_recall = df_mrr_recall.transpose()
df_mrr_recall = df_mrr_recall.rename(columns={0:"MRR", 1:"recall@1"})
df_mrr_recall.to_csv(os.path.join(OUTPUT_DIR, "df_mrr_recall_tc3.csv"))

df_rank = pd.DataFrame(global_results_tc0)/200
df_rank.index = [(x+1) * 5 for x in df_rank.index]
df_rank.to_csv(os.path.join(OUTPUT_DIR, "df_rank_tc1.csv"))

df_rank = pd.DataFrame(global_results_tc1)/200
df_rank.index = [(x+1) * 5 for x in df_rank.index]
df_rank.to_csv(os.path.join(OUTPUT_DIR, "df_rank_tc2.csv"))

df_rank = pd.DataFrame(global_results_tc2)/200
df_rank.index = [(x+1) * 5 for x in df_rank.index]
df_rank.to_csv(os.path.join(OUTPUT_DIR, "df_rank_tc3.csv"))

## AUC

In [17]:
base_path = "../../DBs/Dataset-2/pairs/"

df_pos_testing = pd.read_csv(
    os.path.join(base_path, "pos_testing_Dataset-2.csv"),
    index_col=0)

df_neg_testing = pd.read_csv(
    os.path.join(base_path, "neg_testing_Dataset-2.csv"),
    index_col=0)

global_results = list()

for csv_file in os.listdir(RESULTS_DIR):
    if (not csv_file.endswith(".csv")) or \
        (not "df_pos" in csv_file) or \
        ("rank" in csv_file):
        continue

    print("Processing {}".format(csv_file))

    df_pos_test = pd.read_csv(
        os.path.join(RESULTS_DIR, csv_file),
        index_col=0)

    df_neg_test = pd.read_csv(
        os.path.join(RESULTS_DIR, csv_file.replace("df_pos", "df_neg")),
        index_col=0)
    
    if 'cs' not in set(df_pos_test.columns):
        df_pos_test.rename(columns={'sim':'cs'}, inplace=True)
        df_neg_test.rename(columns={'sim':'cs'}, inplace=True)

    if 'db_type' in df_pos_test.columns:
        del df_pos_test['db_type']

    if 'db_type' in df_neg_test.columns:
        del df_neg_test['db_type']

    # Merge
    df_pos_testing_m = process_df(
        df_pos_testing, df_pos_test, is_pos=True)
    df_neg_testing_m = process_df(
        df_neg_testing, df_neg_test, is_pos=False)
    
#     display(df_pos_test['cs'].hist(bins=200, range=[-2, +2]))
#     display(df_neg_test['cs'].hist(bins=200, alpha=0.8, range=[-2, +2]))
#     plt.show()

    display(df_pos_test['cs'].hist(bins=200))
    display(df_neg_test['cs'].hist(bins=200, alpha=0.8))
    plt.show()

    # Debug
    assert(df_pos_testing_m.isna().sum()['cs'] == 0)
    assert(df_neg_testing_m.isna().sum()['cs'] == 0)

    result_list = list()
    
    result_list.append(['title', csv_file.replace("pos_", "").replace("neg_", "").replace(".csv", "")])

    ll = plot_roc_values(df_pos_testing_m, df_neg_testing_m, plot=False)
    result_list.extend(ll)
    print("\n")

    global_results.append(result_list)


In [18]:
pd_temp_dict = defaultdict(list)
for xr in global_results:
    columns_set = set()
    columns = [x[0].replace(' - all', '') for x in xr]
    values = [x[1].replace(' - all', '') for x in xr]
    for c, v in zip(columns, values):
        if c in columns_set:
            continue
        columns_set.add(c)
        pd_temp_dict[c].append(v)
df_auc = pd.DataFrame.from_dict(pd_temp_dict)
df_auc = df_auc.rename(columns={"title":"model_name",
                                df_auc.columns[1]:"AUC X-OPT",
                                df_auc.columns[2]:"X-ARCH",
                                df_auc.columns[3]:"X-MIXED"})
display(df_auc)
df_auc.to_csv(os.path.join(OUTPUT_DIR, "df_auc.csv"))

IndexError: index 1 is out of bounds for axis 0 with size 0